In [1]:
import backend
import json
import generate_page as page
initial_prompt = "Create a schema for a very simple twitter application"
schema_path = f'generated/schema.json'
schema_page_path= f'test/convex/schema.ts'
actions_path = 'generated/actions.json'
crud_path = 'generated/crud.ts'
backend_json = 'generated/backend.json'
backend_path = 'test/convex/backend.ts'
page_path = 'generated/page_schema.json'
test_path = 'test/app'


In [ ]:
backend.write_to_file(backend.create_schema_structure(initial_prompt), schema_path)

In [ ]:
schema = backend.read(schema_path)
backend.write_to_file(backend.create_actions(f"Create it for the following structure: \n<applicationStructure>\n{schema}\n</applicationStructure>"), actions_path)

In [ ]:
# entirely compiler
schema = json.loads(backend.read(schema_path))
actions = backend.read(actions_path)
schema_page = backend.create_schema(schema, schema_page_path)
crud_page = backend.create_crud(schema, crud_path)
# faker_data = backend.create_faker_data_code(schema, 'generated/faker.ts')
actions_code = backend.create_actions_code(actions, schema_page, crud_page, backend_json)
actions_page = backend.create_actions_page(json.loads(actions), json.loads(actions_code),crud_page, backend_path)

In [4]:
# creating the page structure
schema = backend.read(schema_path)
actions = backend.read(actions_path)
backend.write_to_file(backend.create_page_structure("Task: " + initial_prompt + "\n\nHere is the database schema for the avaiable database objects you are to use: \n<Schemas>\n" + str(schema) + "\n</Schemas>\n\n Here are the backend queries and mutations that you can call: \n<backend>" + str(actions) + "\n</backend>\n\nNow, generate the page layout strictly in the json format described above." ), page_path)

Here is the page layout schema for the simple Twitter application in the specified JSON format:

<jsonSchema>
{
  "description": "A simple Twitter-like application that allows users to post tweets, follow other users, and view a timeline of tweets from followed users.",
  "features_description": "The main pages include the home timeline, user profile, and a page to view a single tweet. The home timeline shows tweets from followed users. The user profile page shows the user's tweets, followers, and following. The single tweet page allows viewing replies.",
  "pages": [
    {
      "route_name": "home",
      "docs": "The home timeline page showing tweets from followed users",
      "page_title": "Home Timeline",
      "description": "View tweets from users you follow",
      "data_calls": [
        {
          "query_name": "getFollowing",
          "purpose": "Get the list of users the current user is following to fetch their tweets",
          "arguments": [
            {
            

In [3]:
# alternative way to make page structure: simplified version
page.write(page_path, page.generate_component_list(schema_path, actions_path))

[
  {
    "url": "/",
    "title": "Home",
    "description": "The main timeline view showing tweets from followed users"
  },
  {
    "url": "/register",
    "title": "Register",
    "description": "User registration page"
  },
  {
    "url": "/login",
    "title": "Login",
    "description": "User login page"
  },
  {
    "url": "/profile/[username]",
    "title": "UserProfile",
    "description": "A user's profile page showing their tweets, followers, and following"
  },
  {
    "url": "/compose",
    "title": "ComposeTweet",
    "description": "Page to compose and post a new tweet"
  }
]
{
    "pages": [
        {
            "url": "/",
            "title": "Home",
            "description": "The main timeline view showing tweets from followed users",
            "components": [
                {
                    "name": "Navbar",
                    "description": "Navigation bar with links to other pages"
                },
                {
                    "name": "Tweet

In [7]:
# generate single page
pages = json.loads(backend.read(page_path))["pages"]
page.generate_page(test_path, pages, pages[0], schema_path, actions_path)


You are writing a web app with Next.js, Convex, and Tailwind. Take the following description of the app and a list of pages and components, and edit the boilerplate file. 
The app description is as follows: 
A simple Twitter-like application that allows users to post tweets, follow other users, and view a timeline of tweets from followed users.
Features: - User registration and login
- Posting tweets
- Following other users
- Timeline view of tweets from followed users
- User profile pages showing their tweets
Actions: The application will need actions to get a user's profile information, get a user's tweets, get the list of users a user is following, get the list of users following a user, post a new tweet, and follow/unfollow another user.
You are to write the Home Timeline page.
List of pages:
[
  {
    "url": "/home",
    "docs": "The home timeline page showing tweets from followed users",
    "title": "Home Timeline",
    "description": "View tweets from users you follow",
    "d